# 浙菜、闽菜相关信息爬取

In [1]:
# 导入所需要的包
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import os
import csv

In [2]:
# 生成请求头
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82",
    "Connection": "keep-alive",
    "Referer": "https://home.meishichina.com/recipe-type.html"
}

### 浙菜

In [3]:
# 生成 urls list
urls_zhecai = ['https://home.meishichina.com/recipe/zhecai/page/{}/'.format(str(i)) for i in range(1, 4)]
data_lst = []
# 遍历每一个列表页
for url_zhecai in urls_zhecai:
    # 封装发送请求并获取响应
    response = requests.get(url=url_zhecai, headers=request_headers)
    # 判断状态码
    status_code = response.status_code
    if status_code == 200:
        html_text = response.text
        # 解析 html 文本
        bs = BeautifulSoup(html_text, "lxml")
        # 定位到详情页链接的 a 标签
        urls_cuisine = bs.select('#J_list > ul > li > div.detail > h2 > a')
        img_infos = bs.select('#J_list > ul > li > div.pic > a > img')
        # 遍历所有菜品
        for cuisine, img_info in zip(urls_cuisine, img_infos):
            # 提取详情页链接，即 href 属性值
            url_cuisine = cuisine.get('href')
            # 由于尝试发现详情页中图片不合适，所以回到列表页抓取图片
            cuisine_name = cuisine.get('title')
            pic_url = img_info.get('data-src')
            resp = requests.get(pic_url, headers=request_headers)
            # 检查响应状态码
            if resp.status_code == 200:
                # 获取图片内容
                image_content = resp.content
                file_name = cuisine_name + ".jpg"
                file_path = os.path.join(r'D:\PYZ\PT(Shi_xi_shi_xun)\Group\menu_tasks\pictures', 'zhe')
                if not os.path.exists(file_path):
                    os.mkdir(file_path)

                # 完成写入
                with open(file_path + os.sep + file_name, 'wb') as fp:
                    fp.write(image_content)

            # 获取每一个菜品详情页面中的信息
            # 封装发送请求并获取响应
            response = requests.get(url=url_cuisine, headers=request_headers)
            # 判断状态码
            status_code = response.status_code
            if status_code == 200:
                # 防止中文乱码，因此自动识别编码方式后再提取文本
                response.encoding = response.apparent_encoding
                html_text = response.text
                # 解析 html 文本
                bs = BeautifulSoup(html_text, "lxml")
                selector = etree.HTML(html_text)
                # 获取菜品名称
                cuisine_name = bs.select('body > div.wrap > div > div.space_left > div.userTop.clear > h1 > a')[0].get_text().strip()
                # print(cuisine_name)
                # 获取菜品主料
                # 悲，本来觉得不难！结果从这第二个数据就卡死了难蚌！太难了！！！最难得是定位啊！！！有规律的网页是正规网页人家会反爬，哭；不反爬的小网页，它可能不规律，淦！
                # 由于使用CSS选择器的定位方式，发现定位不好搞，因为fieldset后面的数字不同，也就是层数不一样（悲），但是！经过桐桐的提醒，发现xpath是有规律的！所以站在巨人的肩膀上干活效率会变高（bushi）
                main_ingredient_dict = {}
                main_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/a/b')
                for i in range(0,len(main_ingredient_names)):
                    main_ingredient_name = main_ingredient_names[i].text
                    main_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[2]')
                    main_ingredient_dict[main_ingredient_name] = main_ingredient_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                main_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/b')
                if len(main_ingredient_names) != 0:
                    for i in range(0,len(main_ingredient_names)):
                        main_ingredient_name = main_ingredient_names[i].text
                        main_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[2]')
                        main_ingredient_dict[main_ingredient_name] = main_ingredient_dosage[i].text
                # print(main_ingredient_dict)

                # 获取菜品辅料
                # 为了避免麻烦，继续采用上面这个方法
                secondary_ingredient_dict = {}
                secondary_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/a/b')
                for i in range(0,len(secondary_ingredient_names)):
                    secondary_ingredient_name = secondary_ingredient_names[i].text
                    secondary_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[2]')
                    secondary_ingredient_dict[secondary_ingredient_name] = secondary_ingredient_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                secondary_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/b')
                if len(secondary_ingredient_names) != 0:
                    for i in range(0,len(secondary_ingredient_names)):
                        secondary_ingredient_name = secondary_ingredient_names[i].text
                        secondary_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[2]')
                        secondary_ingredient_dict[secondary_ingredient_name] = secondary_ingredient_dosage[i].text
                # print(secondary_ingredient_dict)

                # 获取菜品调料
                # 突出一个追求效率（潜台词：懒）
                spices_dict = {}
                spices_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/a/b')
                for i in range(0,len(spices_names)):
                    spices_name = spices_names[i].text
                    spices_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[2]')
                    spices_dict[spices_name] = spices_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                spices_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/b')
                if len(spices_names) != 0:
                    for i in range(0,len(spices_names)):
                        spices_name = spices_names[i].text
                        spices_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[2]')
                        spices_dict[spices_name] = spices_dosage[i].text
                # print(spices_dict)

                # 获取食材明细最后一块 —— 其实应该是特性
                # 口味 工艺 耗时 难度
                detail_info = {
                    '口味': None,
                    '工艺': None,
                    '耗时': None,
                    '难度': None
                }
                index = 1
                for detail_info_key in detail_info:
                    detail = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[4]/ul/li[%d]/span[1]/a/text()'%index)
                    index += 1
                    if len(detail) != 0:
                        detail_info[detail_info_key] = detail[0]
                # print(detail_info)

                # 激动人心的时刻！最重要的东西是做菜的步骤啊！
                method_details = ''
                method_num = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/div')
                method = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/text()')
                for i in range(0,len(method_num)):
                    method_details += method_num[i].text + '、'  + method[i] + '  '
                # print(method_details)
                data_lst.append((cuisine_name,main_ingredient_dict,secondary_ingredient_dict,spices_dict,detail_info,method_details))

# 写到这忽然感觉我不如直接写.py文件，反正似乎也不怎么用分块的调试来着，接下来根据统一的格式设计存入 csv文件 的部分
file_path = os.path.join(r'D:\PYZ\PT(Shi_xi_shi_xun)\Group\menu_tasks\data', 'zhe')
if not os.path.exists(file_path):
    os.mkdir(file_path)

with open(file_path + os.sep + 'dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','口味','工艺','耗时','难度','步骤'])

with open(file_path + os.sep + 'main_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'secondary_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'spices.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'ingredient_amount.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['原料名','用量','菜品名'])

with open(file_path + os.sep + 'cuisine_dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','属于','菜系'])

# 避免重复存储
main_ingredient_have_been_added = []
secondary_ingredient_have_been_added = []
spices_dict_have_been_added = []

for data in data_lst:
    # dish.csv
    with open(file_path + os.sep + 'dish.csv', 'a', newline='',encoding='utf-8') as f: 
        writer = csv.writer(f)
        writer.writerow([data[0],data[4]['口味'],data[4]['工艺'],data[4]['耗时'],data[4]['难度'],data[5]])
    # main_ingredient.csv
    with open(file_path + os.sep + 'main_ingredient.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for main_ingredient_name in data[1]:
                if main_ingredient_name in main_ingredient_have_been_added:
                    continue
                main_ingredient_have_been_added.append(main_ingredient_name)
                writer.writerow([main_ingredient_name])
    # secondary_ingredient.csv            
    with open(file_path + os.sep + 'secondary_ingredient.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for secondary_ingredient_name in data[2]:
            if secondary_ingredient_name in secondary_ingredient_have_been_added:
                continue
            secondary_ingredient_have_been_added.append(secondary_ingredient_name)
            writer.writerow([secondary_ingredient_name])
            
                
    with open(file_path + os.sep + 'spices.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for spices_name in data[3]:
            if spices_name in spices_dict_have_been_added:
                continue
            spices_dict_have_been_added.append(spices_name)
            writer.writerow([spices_name])
                

    with open(file_path + os.sep + 'ingredient_amount.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for main_ingredient_name in data[1]:
            writer.writerow([main_ingredient_name,data[1][main_ingredient_name],data[0]])
        for secondary_ingredient_name in data[2]:
            writer.writerow([secondary_ingredient_name,data[2][secondary_ingredient_name],data[0]])
        for spices_name in data[3]:
            writer.writerow([spices_name,data[3][spices_name],data[0]])

    with open(file_path + os.sep + 'cuisine_dish.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([data[0],'属于','浙菜'])
print('END...')

END...


### 闽菜

In [4]:
# 生成 urls list
urls_mincai = ['https://home.meishichina.com/recipe/mincai/page/{}/'.format(str(i)) for i in range(1, 4)]
data_lst = []
# 遍历每一个列表页
for url_mincai in urls_mincai:
    # 封装发送请求并获取响应
    response = requests.get(url=url_mincai, headers=request_headers)
    # 判断状态码
    status_code = response.status_code
    if status_code == 200:
        html_text = response.text
        # 解析 html 文本
        bs = BeautifulSoup(html_text, "lxml")
        # 定位到详情页链接的 a 标签
        urls_cuisine = bs.select('#J_list > ul > li > div.detail > h2 > a')
        img_infos = bs.select('#J_list > ul > li > div.pic > a > img')
        # 遍历所有菜品
        for cuisine, img_info in zip(urls_cuisine, img_infos):
            # 提取详情页链接，即 href 属性值
            url_cuisine = cuisine.get('href')
            # 由于尝试发现详情页中图片不合适，所以回到列表页抓取图片
            cuisine_name = cuisine.get('title')
            pic_url = img_info.get('data-src')
            resp = requests.get(pic_url, headers=request_headers)
            # 检查响应状态码
            if resp.status_code == 200:
                # 获取图片内容
                image_content = resp.content
                file_name = cuisine_name + ".jpg"
                file_path = os.path.join(r'D:\PYZ\PT(Shi_xi_shi_xun)\Group\menu_tasks\pictures', 'min')
                if not os.path.exists(file_path):
                    os.mkdir(file_path)

                # 完成写入
                with open(file_path + os.sep + file_name, 'wb') as fp:
                    fp.write(image_content)

            # 获取每一个菜品详情页面中的信息
            # 封装发送请求并获取响应
            response = requests.get(url=url_cuisine, headers=request_headers)
            # 判断状态码
            status_code = response.status_code
            if status_code == 200:
                # 防止中文乱码，因此自动识别编码方式后再提取文本
                response.encoding = response.apparent_encoding
                html_text = response.text
                # 解析 html 文本
                bs = BeautifulSoup(html_text, "lxml")
                selector = etree.HTML(html_text)
                # 获取菜品名称
                cuisine_name = bs.select('body > div.wrap > div > div.space_left > div.userTop.clear > h1 > a')[0].get_text().strip()
                # print(cuisine_name)
                # 获取菜品主料
                # 悲，本来觉得不难！结果从这第二个数据就卡死了难蚌！太难了！！！最难得是定位啊！！！有规律的网页是正规网页人家会反爬，哭；不反爬的小网页，它可能不规律，淦！
                # 由于使用CSS选择器的定位方式，发现定位不好搞，因为fieldset后面的数字不同，也就是层数不一样（悲），但是！经过桐桐的提醒，发现xpath是有规律的！所以站在巨人的肩膀上干活效率会变高（bushi）
                main_ingredient_dict = {}
                main_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/a/b')
                for i in range(0,len(main_ingredient_names)):
                    main_ingredient_name = main_ingredient_names[i].text
                    main_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[2]')
                    main_ingredient_dict[main_ingredient_name] = main_ingredient_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                main_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/b')
                if len(main_ingredient_names) != 0:
                    for i in range(0,len(main_ingredient_names)):
                        main_ingredient_name = main_ingredient_names[i].text
                        main_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[2]')
                        main_ingredient_dict[main_ingredient_name] = main_ingredient_dosage[i].text
                # print(main_ingredient_dict)

                # 获取菜品辅料
                # 为了避免麻烦，继续采用上面这个方法
                secondary_ingredient_dict = {}
                secondary_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/a/b')
                for i in range(0,len(secondary_ingredient_names)):
                    secondary_ingredient_name = secondary_ingredient_names[i].text
                    secondary_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[2]')
                    secondary_ingredient_dict[secondary_ingredient_name] = secondary_ingredient_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                secondary_ingredient_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/b')
                if len(secondary_ingredient_names) != 0:
                    for i in range(0,len(secondary_ingredient_names)):
                        secondary_ingredient_name = secondary_ingredient_names[i].text
                        secondary_ingredient_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[2]')
                        secondary_ingredient_dict[secondary_ingredient_name] = secondary_ingredient_dosage[i].text
                # print(secondary_ingredient_dict)

                # 获取菜品调料
                # 突出一个追求效率（潜台词：懒）
                spices_dict = {}
                spices_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/a/b')
                for i in range(0,len(spices_names)):
                    spices_name = spices_names[i].text
                    spices_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[2]')
                    spices_dict[spices_name] = spices_dosage[i].text
                # 这里是处理没有链接的那种情况，两种定位不一样
                spices_names = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/b')
                if len(spices_names) != 0:
                    for i in range(0,len(spices_names)):
                        spices_name = spices_names[i].text
                        spices_dosage = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[2]')
                        spices_dict[spices_name] = spices_dosage[i].text
                # print(spices_dict)

                # 获取食材明细最后一块 —— 其实应该是特性
                # 口味 工艺 耗时 难度
                detail_info = {
                    '口味': None,
                    '工艺': None,
                    '耗时': None,
                    '难度': None
                }
                index = 1
                for detail_info_key in detail_info:
                    detail = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[4]/ul/li[%d]/span[1]/a/text()'%index)
                    index += 1
                    if len(detail) != 0:
                        detail_info[detail_info_key] = detail[0]
                # print(detail_info)

                # 激动人心的时刻！最重要的东西是做菜的步骤啊！
                method_details = ''
                method_num = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/div')
                method = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/text()')
                for i in range(0,len(method_num)):
                    method_details += method_num[i].text + '、' + method[i] + '  '
                # print(method_details)
                data_lst.append((cuisine_name,main_ingredient_dict,secondary_ingredient_dict,spices_dict,detail_info,method_details))

# 写到这忽然感觉我不如直接写.py文件，反正似乎也不怎么用分块的调试来着，接下来根据统一的格式设计存入 csv文件 的部分
file_path = os.path.join(r'D:\PYZ\PT(Shi_xi_shi_xun)\Group\menu_tasks\data', 'min')
if not os.path.exists(file_path):
    os.mkdir(file_path)

with open(file_path + os.sep + 'dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','口味','工艺','耗时','难度','步骤'])

with open(file_path + os.sep + 'main_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'secondary_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'spices.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open(file_path + os.sep + 'ingredient_amount.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['原料名','用量','菜品名'])

with open(file_path + os.sep + 'cuisine_dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','属于','菜系'])

# 避免重复存储
main_ingredient_have_been_added = []
secondary_ingredient_have_been_added = []
spices_dict_have_been_added = []

for data in data_lst:
    # dish.csv
    with open(file_path + os.sep + 'dish.csv', 'a', newline='',encoding='utf-8') as f: 
        writer = csv.writer(f)
        writer.writerow([data[0],data[4]['口味'],data[4]['工艺'],data[4]['耗时'],data[4]['难度'],data[5]])
    # main_ingredient.csv
    with open(file_path + os.sep + 'main_ingredient.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for main_ingredient_name in data[1]:
                if main_ingredient_name in main_ingredient_have_been_added:
                    continue
                main_ingredient_have_been_added.append(main_ingredient_name)
                writer.writerow([main_ingredient_name])
    # secondary_ingredient.csv            
    with open(file_path + os.sep + 'secondary_ingredient.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for secondary_ingredient_name in data[2]:
            if secondary_ingredient_name in secondary_ingredient_have_been_added:
                continue
            secondary_ingredient_have_been_added.append(secondary_ingredient_name)
            writer.writerow([secondary_ingredient_name])
            
                
    with open(file_path + os.sep + 'spices.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for spices_name in data[3]:
            if spices_name in spices_dict_have_been_added:
                continue
            spices_dict_have_been_added.append(spices_name)
            writer.writerow([spices_name])
                

    with open(file_path + os.sep + 'ingredient_amount.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        for main_ingredient_name in data[1]:
            writer.writerow([main_ingredient_name,data[1][main_ingredient_name],data[0]])
        for secondary_ingredient_name in data[2]:
            writer.writerow([secondary_ingredient_name,data[2][secondary_ingredient_name],data[0]])
        for spices_name in data[3]:
            writer.writerow([spices_name,data[3][spices_name],data[0]])

    with open(file_path + os.sep + 'cuisine_dish.csv','a',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([data[0],'属于','闽菜'])
print('END...')

END...
